In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144
from datetime import datetime
import dill

In [2]:
from static_grader import grader

# The New York Social Graph


[New York Social Diary](http://www.newyorksocialdiary.com/) provides a
fascinating lens onto New York's socially well-to-do.  The data forms a natural social graph for New York's social elite.  Take a look at this page of a recent [run-of-the-mill holiday party](http://www.newyorksocialdiary.com/party-pictures/2014/holiday-dinners-and-doers).

Besides the brand-name celebrities, you will notice the photos have carefully annotated captions labeling those that appear in the photos.  We can think of this as implicitly implying a social graph: there is a connection between two individuals if they appear in a picture together.

For this project, we will assemble the social graph from photo captions for parties dated December 1, 2014, and before.  Using this graph, we can make guesses at the most popular socialites, the most influential people, and the most tightly coupled pairs.

We will attack the project in three phases:
1. Get a list of all the photo pages to be analyzed.
2. Parse all of the captions on a sample page.
3. Parse all of the captions on all pages, and assemble the graph.

## Phase One


The first step is to crawl the data.  We want photos from parties on or before December 1st, 2014.  Go to the [Party Pictures Archive](http://www.newyorksocialdiary.com/party-pictures) to see a list of (party) pages.  We want to get the url for each party page, along with its date.

Here are some packages that you may find useful.  You are welcome to use others, if you prefer.

In [3]:
import requests
import dill
from bs4 import BeautifulSoup
from datetime import datetime

We recommend using Python [Requests](http://docs.python-requests.org/en/master/) to download the HTML pages, and [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) to process the HTML.  Let's start by getting the [first page](http://www.newyorksocialdiary.com/party-pictures).

In [4]:
page = requests.get("https://web.archive.org/web/20150913224145/http://www.newyorksocialdiary.com/party-pictures/") # Use requests.get to download the page.
page.status_code

200

Now, we process the text of the page with BeautifulSoup.

In [5]:
soup = BeautifulSoup(page.text, "lxml")
soup.prettify()

'<!DOCTYPE html>\n<!--[if IEMobile 7]><html class="no-js ie iem7" lang="en" dir="ltr"><![endif]-->\n<!--[if lte IE 6]><html class="no-js ie lt-ie9 lt-ie8 lt-ie7" lang="en" dir="ltr"><![endif]-->\n<!--[if (IE 7)&(!IEMobile)]><html class="no-js ie lt-ie9 lt-ie8" lang="en" dir="ltr"><![endif]-->\n<!--[if IE 8]><html class="no-js ie lt-ie9" lang="en" dir="ltr"><![endif]-->\n<!--[if (gte IE 9)|(gt IEMobile 7)]><html class="no-js ie" lang="en" dir="ltr" prefix="fb: http://ogp.me/ns/fb# og: http://ogp.me/ns# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/profile# video: http://ogp.me/ns/video# product: http://ogp.me/ns/product#"><![endif]-->\n<!--[if !IE]><!-->\n<html class="no-js" dir="ltr" lang="en" prefix="fb: http://ogp.me/ns/fb# og: http://ogp.me/ns# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/profile# video: http://ogp.me/ns/video# product: http://ogp.me/ns/product#">\n <!--<![endif]-->\n <head>\n  

This page has links to 50 party pages. Look at the structure of the page and determine how to isolate those links.  Your browser's developer tools (usually `Cmd`-`Option`-`I` on Mac, `Ctrl`-`Shift`-`I` on others) offer helpful tools to explore the structure of the HTML page.

Once you have found a pattern, use BeautifulSoup's [select](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors) or [find_all](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find) methods to get those elements.

In [6]:
links = soup.find_all('div', attrs={'class':'views-row'})
len(links)

50

There should be 50 per page.

In [7]:
assert len(links) == 50

Let's take a look at that first link.  Figure out how to extract the URL of the link, as well as the date.  You probably want to use `datetime.strptime`.  See the [format codes for dates](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) for reference.

In [8]:
link = links[0]
# Check that the title and date match what you see visually.
link.prettify()

'<div class="views-row views-row-1 views-row-odd views-row-first">\n <span class="views-field views-field-title">\n  <span class="field-content">\n   <a href="/web/20150913224145/http://www.newyorksocialdiary.com/party-pictures/2015/kicks-offs-sing-offs-and-pro-ams">\n    Kicks offs, sing offs, and pro ams\n   </a>\n  </span>\n </span>\n <span class="views-field views-field-created">\n  <span class="field-content">\n   Friday, September 11, 2015\n  </span>\n </span>\n</div>\n'

For purposes of code reuse, let's put that logic into a function.  It should take the link element and return the URL and date parsed from it.

In [9]:
def get_link_date(el):
    url = el.select('a')[0]['href']
    date = datetime.strptime(el.select('span')[-1].text, "%A, %B %d, %Y")
    return url, date

You may want to check that it works as you expected.

Once that's working, let's write another function to parse all of the links on a page.  Thinking ahead, we can make it take a Requests [Response](http://docs.python-requests.org/en/master/api/#requests.Response) object and do the BeautifulSoup parsing within it.

In [10]:
def get_links(response):
    soup = BeautifulSoup(response.text, "lxml")
    links = soup.find_all('div', attrs={'class':'views-row'})
    return list(get_link_date(link) for link in links) # A list of URL, date pairs

If we run this on the previous response, we should get 50 pairs.

In [11]:
assert len(get_links(page)) == 50

But we only want parties with dates on or before the first of December, 2014.  Let's write a function to filter our list of dates to those at or before a cutoff.  Using a keyword argument, we can put in a default cutoff, but allow us to test with others.

In [12]:
def filter_by_date(links, cutoff=datetime(2014, 12, 1)):
    # Return only the elements with date <= cutoff
    return list(filter(lambda x: x[1] <= cutoff, links))

With the default cutoff, there should be no valid parties on the first page.  Adjust the cutoff date to check that it is actually working.

In [13]:
assert len(filter_by_date(get_links(page))) == 0

Now we should be ready to get all of the party URLs.  Click through a few of the index pages to determine how the URL changes.  Figure out a strategy to visit all of them.

HTTP requests are generally IO-bound.  This means that most of the time is spent waiting for the remote server to respond.  If you use `requests` directly, you can only wait on one response at a time.  [requests-futures](https://github.com/ross/requests-futures) lets you wait for multiple requests at a time.  You may wish to use this to speed up the downloading process.

In [14]:
from requests_futures.sessions import FuturesSession

link_list = []
# You can use link_list.extend(others) to add the elements of others
# to link_list.
LIMIT = 26
def get_page_args(i):
    return {"url": "https://web.archive.org/web/20150913224145/http://www.newyorksocialdiary.com/party-pictures/",
            "params": {"page": i}}

session = FuturesSession(max_workers=5)
futures = [session.get(**get_page_args(i)) for i in range(1, LIMIT)]

for future in futures:
    link_list.extend(filter_by_date(get_links(future.result())))

In the end, you should have 1193 parties.

In [15]:
assert len(link_list) == 1193

In case we need to restart the notebook, we should save this information to a file.  There are many ways you could do this; here's one using `dill`.

In [16]:
dill.dump(link_list, open('nysd-links.pkd', 'wb'))

To restore the list, we can just load it from the file.  When the notebook is restarted, you can skip the code above and just run this command.

In [17]:
link_list = dill.load(open('nysd-links.pkd', 'rb'))

## Question 1: histogram


Get the number of party pages for the 95 months (that is, month-year pair) in the data.  Represent this histogram as a list of 95 tuples, each of the form `("Dec-2014", 1)`.  Note that you can convert `datetime` objects into these sort of strings with `strftime` and the [format codes for dates](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior).

Plot the histogram for yourself.  Do you see any trends?

In [18]:
from collections import Counter

date_ls = Counter(list(link[1].strftime("%b-%Y") for link in link_list))
histogram = list(zip(date_ls.keys(), date_ls.values()))

In [19]:
grader.score('graph__histogram', histogram)

Your score:  0.9789473684210537


## Phase Two


In this phase, we we concentrate on getting the names out of captions for a given page.  We'll start with [the benefit cocktails and dinner](http://www.newyorksocialdiary.com/party-pictures/2015/celebrating-the-neighborhood) for [Lenox Hill Neighborhood House](http://www.lenoxhill.org/), a neighborhood organization for the East Side.

Take a look at that page.  Note that some of the text on the page is captions, but others are descriptions of the event.  Determine how to select only the captions.

In [36]:
page = requests.get("https://web.archive.org/web/20151114014941/http://www.newyorksocialdiary.com/party-pictures/2015/celebrating-the-neighborhood")
soup = BeautifulSoup(page.text, "lxml")
captions = [caption.text for caption in soup.find_all(attrs={'class':'photocaption'})]

By our count, there are about 110.  But if you're off by a couple, you're probably okay.

In [37]:
assert abs(len(captions) - 110) < 5

Let's encapsulate this in a function.  As with the links pages, we want to avoid downloading a given page the next time we need to run the notebook.  While we could save the files by hand, as we did before, a checkpoint library like [ediblepickle](https://pypi.python.org/pypi/ediblepickle/1.1.3) can handle this for you.  (Note, though, that you may not want to enable this until you are sure that your function is working.)

You should also keep in mind that HTTP requests fail occasionally, for transient reasons.  You should plan how to detect and react to these failures.   The [retrying module](https://pypi.python.org/pypi/retrying) is one way to deal with this.

In [45]:
import os

cache_dir = 'cache'
if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)
    
def process_link(page):
    soup = BeautifulSoup(page.text, "lxml")
    captions = [caption.text for caption in soup.find_all(attrs={'class':'photocaption'})]
    return captions

def get_captions(path):
#     if os.path.exists('cache/' + path)
    captions = []
    if isinstance(path, str):
        path = [path]
    # multiprocessing
    base_url = "https://web.archive.org/web/20151114014941/http://www.newyorksocialdiary.com"
    session = FuturesSession(max_workers=5)
    futures = [session.get(base_url + p) for p in path]
    # page = requests.get("https://web.archive.org/web/20151114014941/http://www.newyorksocialdiary.com" + path)
#     soup = BeautifulSoup(page.text, "lxml")
#     captions = soup.find_all(attrs={'class':'photocaption'})
    for future in futures:
        captions.extend(process_link(future.result()))
    return captions

This should get the same captions as before.

In [47]:
assert captions == get_captions("/party-pictures/2015/celebrating-the-neighborhood")

Now that we have some sample captions, let's start parsing names out of those captions.  There are many ways of going about this, and we leave the details up to you.  Some issues to consider:

  1. Some captions are not useful: they contain long narrative texts that explain the event.  Try to find some heuristic rules to separate captions that are a list of names from those that are not.  A few heuristics include:
    - look for sentences (which have verbs) and as opposed to lists of nouns. For example, [`nltk` does part of speech tagging](http://www.nltk.org/book/ch05.html) but it is a little slow. There may also be heuristics that accomplish the same thing.
    - Similarly, spaCy's [entity recognition](https://spacy.io/docs/usage/entity-recognition) could be useful here.
    - Look for commonly repeated threads (e.g. you might end up picking up the photo credits or people such as "a friend").
    - Long captions are often not lists of people.  The cutoff is subjective, but for grading purposes, *set that cutoff at 250 characters*.
  2. You will want to separate the captions based on various forms of punctuation.  Try using `re.split`, which is more sophisticated than `string.split`. **Note**: The reference solution uses regex exclusively for name parsing.
  3. You might find a person named "ra Lebenthal".  There is no one by this name.  Can anyone spot what's happening here?
  4. This site is pretty formal and likes to say things like "Mayor Michael Bloomberg" after his election but "Michael Bloomberg" before his election.  Can you find other ('optional') titles that are being used?  They should probably be filtered out because they ultimately refer to the same person: "Michael Bloomberg."
  5. There is a special case you might find where couples are written as eg. "John and Mary Smith". You will need to write some extra logic to make sure this properly parses to two names: "John Smith" and "Mary Smith".
  6. When parsing names from captions, it can help to look at your output frequently and address the problems that you see coming up, iterating until you have a list that looks reasonable. This is the approach used in the reference solution. Because we can only asymptotically approach perfect identification and entity matching, we have to stop somewhere.
  
**Questions worth considering:**
  1. Who is Patrick McMullan and should he be included in the results? How would you address this?
  2. What else could you do to improve the quality of the graph's information?

In [208]:
text = captions[0].text
text

"Glenn Adamson, Simon Doonan, Victoire de Castellane, Craig Leavitt, Jerome Chazen, Andi Potamkin, Ralph Pucci, Kirsten Bailey, Edwin Hathaway, and Dennis Freedman at the Museum of Art and Design's annual MAD BALL. "

In [56]:
import re

def strip_names(text):
    name_ls = []
    sentence = [s.strip() for s in text.split('.')]
    
    for s in sentence:
        # fisrt last
        pattern = re.findall(r"([A-Z][a-z]+) ([A-Z][A-Za-z\-]+)", s)
        name_ls.extend([p for p in pattern])

        # first1 and first2 last
        pattern = re.findall(r"^([A-Z]*)([A-Z][a-z]+) and ([A-Z]*)([A-Z][a-z]+) ([A-Z][A-Za-z\-]+)", s)
        name_ls.extend([(p[1], p[-1]) for p in pattern])

        # first middle last
        pattern = re.findall(r"([A-Z][a-z]\w+) ([A-Z][a-z]+) ([A-Z][A-Za-z\-]+)", s)
        name_ls.extend([p for p in pattern])
    #     for p in pattern:
    #         if (p[0] in ['Mayor']) or (p[0][-1] == '.'):
    #             name_ls.extend([(p[1], p[2])])
    #         else:
    #             name_ls.extend([p])

        # first special last
        pattern = re.findall(r"([A-Z][a-z]+) de ([A-Z][a-z]\w+)", s)
        name_ls.extend([(p[0], 'de', p[1]) for p in pattern])
    
    return [' '.join(name) for name in name_ls]

def delete_duplicate(ls):
    # excluding partial names
    for name in ls:
        if len(name.split()) == 3:
            first_two = ' '.join(name.split()[:2])
            second_two = ' '.join(name.split()[1:])
            #third_two = name.split()[0] + ' ' + name.split()[2]
            for dup in [first_two, second_two]:
                if dup in ls:
                    index = ls.index(dup)
                    ls.pop(index)

    # excluding non-people names and Pre-fix
    new_ls = []
    for i in range(len(ls)):
        name = ls[i]
        # remove some unusual words
        remove_ls = ['New York', 'Director', 'Member', 'Gala', 'The', 'Row', \
                     'Committee', 'President', 'Special', 'Los Angeles', 'Tory Burch', \
                    'City', 'New']
        regular = True
        for w in remove_ls:
            if name.find(w) != -1:
                regular = False
                break

        if regular:
            if name.split()[0] in ['Mayor', 'Dr.', 'Prof.']:
                new_ls.append(' '.join(name.split()[1:]))
            else:
                new_ls.append(name)
    
    return new_ls

In [57]:
text = r"Dustin Hoffman, Jack NiCholson, Mark Chris, Christopher Spitzmiller and Diana Quasha. \
        Jack and Mary Nicholson. Second Row: Mark Chris, Christopher and Diana de Quasha, \
        Jason and Diana de'Quasha. Patrick McMullen, Patrick and Jane McMullen. The Patrick \
        and Jane de McMullen. The Park with Jane de'McMullen. Jackie Chan with Jet Li. \
        Winston and Margaret Thatcher-Churchill"
#text = 'Winston and Margaret Thatcher-Churchill'
delete_duplicate(strip_names(text))

['Dustin Hoffman',
 'Jack NiCholson',
 'Mark Chris',
 'Christopher Spitzmiller',
 'Diana Quasha',
 'Mary Nicholson',
 'Jack Nicholson',
 'Mark Chris',
 'Diana de Quasha',
 'Patrick McMullen',
 'Jane McMullen',
 'Jane de McMullen',
 'Jackie Chan',
 'Jet Li',
 'Margaret Thatcher-Churchill',
 'Winston Thatcher-Churchill']

In [324]:
test = ['Mayor Michael', 'Michael Bloomberg', 'Mayor Michael Bloomberg']
delete_duplicate(test)

['Michael Bloomberg']

## Question 2: sample_names


Once you feel that your algorithm is working well on these captions, parse all of the captions and extract all the names mentioned.  Sort them alphabetically, by first name, and return the first hundred.

In [58]:
def get_caption_names(caption):
    names_ls = []
    names_ls.extend(strip_names(caption))
    names_ls = list(set(names_ls))
    names_ls = delete_duplicate(names_ls)
    return names_ls

sample_names = []
for caption in captions:
    if len(caption) < 250:
        sample_names.extend(get_caption_names(caption))
sample_names = list(set(sample_names))
sample_names.sort()
sample_names
grader.score('graph__sample_names', sample_names[:100])

Your score:  0.9300000000000006


Now, run this sort of test on a few other pages.  You will probably find that other pages have a slightly different HTML structure, as well as new captions that trip up your caption parser.  But don't worry if the parser isn't perfect -- just try to get the easy cases.

## Phase Three


Once you are satisfied that your caption scraper and parser are working, run this for all of the pages.  If you haven't implemented some caching of the captions, you probably want to do this first.

In [48]:
%%time
# Scraping all of the pages could take 10 minutes or so.
path = [link[0].split('http://www.newyorksocialdiary.com')[-1] for link in link_list]
caption_ls = get_captions(path)
# for link in link_list:
#     path = link[0].split('http://www.newyorksocialdiary.com')[-1]
#     caption_ls.extend(get_captions(path))
#     print(path)
    
len(caption_ls)

CPU times: user 1min 8s, sys: 1.02 s, total: 1min 9s
Wall time: 3min 59s


In [50]:
dill.dump(caption_ls, open('captions.pkd', 'wb'))

For the remaining analysis, we think of the problem in terms of a
[network](http://en.wikipedia.org/wiki/Computer_network) or a
[graph](https://en.wikipedia.org/wiki/Graph_%28discrete_mathematics%29).  Any time a pair of people appear in a photo together, that is considered a link.  What we have described is more appropriately called an (undirected)
[multigraph](http://en.wikipedia.org/wiki/Multigraph) with no self-loops but this has an obvious analog in terms of an undirected [weighted graph](http://en.wikipedia.org/wiki/Graph_%28mathematics%29#Weighted_graph).  In this problem, we will analyze the social graph of the New York social elite.  We recommend using python's [`networkx`](https://networkx.github.io/) library.

In [51]:
import itertools  # itertools.combinations may be useful
import networkx as nx

In [93]:
graph = nx.Graph()

In [96]:
graph.add_edge('Bob', 'Pete')
graph.add_edge('Pete', 'Kevin')

In [97]:
nx.MultiGraph()

In [67]:
len(graph.degree())

90554

All in all, you should end up with over 100,000 captions and more than 110,000 names, connected in about 200,000 pairs.

## Question 3: degree


The simplest question to ask is "who is the most popular"?  The easiest way to answer this question is to look at how many connections everyone has.  Return the top 100 people and their degree.  Remember that if an edge of the graph has weight 2, it counts for 2 in the degree.

**Checkpoint:** Some aggregate stats on the solution

    "count": 100.0
    "mean": 189.92
    "std": 87.8053034454
    "min": 124.0
    "25%": 138.0
    "50%": 157.0
    "75%": 195.0
    "max": 666.0

In [59]:
def create_edge(caption):
    names_ls = get_caption_names(caption)
    for i in range(len(names_ls)):
        for j in range(i+1, len(names_ls)):
            graph.add_edge(names_ls[i], names_ls[j])

graph = nx.Graph()
for caption in caption_ls:
    create_edge(caption)

In [66]:
import heapq  
from operator import itemgetter

graph_ls = list(graph.degree())
degree = [x for x in heapq.nlargest(100,graph_ls,key=itemgetter(1))]

grader.score('graph__degree', degree)

Your score:  0.8600000000000005


In [68]:
[x for x in heapq.nlargest(1000,graph_ls,key=itemgetter(1))]

[('Jean Shafiroff', 448),
 ('Patrick McMullan', 348),
 ('Mark Gilbertson', 346),
 ('Alexandra Lebenthal', 263),
 ('Gillian Miniter', 243),
 ('Geoffrey Bradfield', 223),
 ('Mario Buatta', 221),
 ('Michael Bloomberg', 217),
 ('Yaz Hernandez', 206),
 ('Eleanora Kennedy', 194),
 ('Kamie Lightburn', 191),
 ('Alina Cho', 190),
 ('Lucia Hwong Gordon', 189),
 ('Somers Farkas', 185),
 ('Andrew Saffir', 183),
 ('Allison Aston', 180),
 ('Sharon Bush', 177),
 ('Debbie Bancroft', 170),
 ('Muffie Potter Aston', 168),
 ('Karen Klopp', 163),
 ('Barbara Tober', 156),
 ('Jamee Gregory', 150),
 ('Bettina Zilkha', 148),
 ('Martha Stewart', 146),
 ('Leonard Lauder', 146),
 ('Donna Karan', 146),
 ('Daniel Boulud', 145),
 ('Amy Fine Collins', 144),
 ('Paula Zahn', 144),
 ('Deborah Norville', 141),
 ('Karen LeFrak', 141),
 ('Lydia Fenet', 139),
 ('Christopher Hyland', 139),
 ('Jennifer Creel', 137),
 ('Nicole Miller', 133),
 ('Margo Langenberg', 130),
 ('Liliana Cavendish', 130),
 ('Bonnie Comley', 128),
 ('A

## Question 4: PageRank


A similar way to determine popularity is to look at their
[PageRank](http://en.wikipedia.org/wiki/PageRank).  PageRank is used for web ranking and was originally
[patented](http://patft.uspto.gov/netacgi/nph-Parser?patentnumber=6285999) by Google and is essentially the stationary distribution of a [Markov
chain](http://en.wikipedia.org/wiki/Markov_chain) implied by the social graph. You can implement this yourself or use the version in `networkx`

Use 0.85 as the damping parameter so that there is a 15% chance of jumping to another vertex at random.

**Checkpoint:** Some aggregate stats on the solution

    "count": 100.0
    "mean": 0.0001841088
    "std": 0.0000758068
    "min": 0.0001238355
    "25%": 0.0001415028
    "50%": 0.0001616183
    "75%": 0.0001972663
    "max": 0.0006085816

In [338]:
G = graph
pagerank = dict(nx.pagerank(G, alpha = .85))
pagerank = heapq.nlargest(100, pagerank.items(), key=lambda i: i[1])

grader.score('graph__pagerank', pagerank)

Your score:  0.9200000000000006


## Question 5: best_friends


Another interesting question is who tend to co-occur with each other.  Give us the 100 edges with the highest weights.

Google these people and see what their connection is.  Can we use this to detect instances of infidelity?

**Checkpoint:** Some aggregate stats on the solution

    "count": 100.0
    "mean": 25.84
    "std": 16.0395470855
    "min": 14.0
    "25%": 16.0
    "50%": 19.0
    "75%": 29.25
    "max": 109.0

In [359]:
def create_weighted_edge(caption, graph):
    names_ls = get_caption_names(caption)
    for i in range(len(names_ls)):
        for j in range(i+1, len(names_ls)):
            if graph.has_edge(names_ls[i], names_ls[j]):
                graph[names_ls[i]][names_ls[j]]['weight'] += 1
            else:
                graph.add_edge(names_ls[i], names_ls[j], weight=1)

graph = nx.Graph()
for caption in caption_ls:
    create_weighted_edge(caption, graph)

In [360]:
graph_ls = [(k, v['weight']) for k, v in dict(graph.edges()).items()]
best_friends = [x for x in heapq.nlargest(100, graph_ls, key=itemgetter(1))]
best_friends
grader.score('graph__best_friends', best_friends)

Your score:  0.8800000000000006


*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*